In [120]:
# importing all the necessary packages and data
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np # Numerical computation package
from sklearn.neural_network import MLPRegressor # multilayer perceptron regressor and classifier
from sklearn.preprocessing import StandardScaler # helper to scale our data
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error

In [121]:
# important!! use index_col=0 to read df otherwise pca starts at col 58
ped = pd.read_csv("data_clean_with_dummies.csv", index_col=0)

# beginning of the code, so everyone has the same
np.random.seed(1) # Set the random seed for reproduceability


# Define a new X with the squared feature and choose only columns that we need for enough variance
X = np.array(ped[ped.columns[57:]])

# Output to predict
y = ped["pedestrians count"]


# Split the dataset into train and test sets
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y, test_size=0.3, random_state=72)

In [122]:
# initialize the neural network to predict the pedestrians given our different features
# use three hidden layers, with sizes 32, 64, and 32, ReLU activation functions,
# stochastic gradient descent optimization and a regularization paramater
# (lambda or alpha) of 0.001, batchsizes of 32 and 1000 epochs
nnet = MLPRegressor(hidden_layer_sizes=(32, 64, 32), activation="relu", solver="sgd",
                    alpha=0.001, max_iter=1000)

In [123]:
# scale the data so it is less sensitive to feature scaling
scaler = StandardScaler()
# scale the inputs
scaler.fit(Xtrain)
Xtrain = scaler.transform(Xtrain)
# apply the same transformation to the test data to have meaningful results
Xtest = scaler.transform(Xtest)

# scale the y different because we can't use standardscaler to a 1d array
# scale the output
mu, sigma = y.mean(), y.std() # We will use this to scale back to original values!
ytest = (ytest - mu) / sigma
ytrain = (ytrain - mu) /sigma

In [125]:
# Fit the network to the train data
nnet.fit(Xtrain, ytrain)

# Make predictions
ypred = nnet.predict(Xtest)

# Reconstruct outputs and scale predictions
ytest = ytest * sigma + mu
ytrain = ytrain * sigma + mu
ypred = ypred * sigma + mu


# Compute the MAE
mae = mean_absolute_error(ytest, ypred)
# Comute R^2
r2 = r2_score(ytest, ypred)

print(f"The mean absolute error is {mae}")
print(f"The R^2 is {r2}")

c:\Users\janag\miniconda3\envs\dsf\lib\site-packages\sklearn\neural_network\_base.py:174: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
c:\Users\janag\miniconda3\envs\dsf\lib\site-packages\sklearn\utils\extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
c:\Users\janag\miniconda3\envs\dsf\lib\site-packages\sklearn\utils\extmath.py:152: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
c:\Users\janag\miniconda3\envs\dsf\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:709: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


ValueError: Solver produced non-finite parameter weights. The input data may contain large values and need to be preprocessed.

The mean absolute error is 525.9776290853022
The R^2 is 0.928619546779347

In [ ]:
# plot the results
fig, ax = plt.subplots(figsize=(16, 12))

#create a linspace to match the y-values
xs = np.linspace(0, ytest.shape[0], num=ytest.shape[0])

# plot the true values for 2 pca variables
ax.scatter(xs, ytest, label="true values", alpha=0.8)

# plot the predictions
ax.scatter(xs, ypred, label="predictions", color = "red", alpha=0.2)

# Add labels and legend
ax.set_xlabel("Features")
ax.set_ylabel("Pedestrians")
ax.legend()

In [ ]:
# scale the data so it is less sensitive to feature scaling
scaler = StandardScaler()
# scale the inputs
scaler.fit(X)
X = scaler.transform(X)

# scale the y different because we can't use standardscaler to a 1d array
mu, sigma = y.mean(), y.std() # We will use this to scale back to original values!
y = (y - mu) / sigma

# create custom scoring function
#def scorer(estimator, x, y):
    #ypred = estimator.predict(x)
    #mae = np.sum(np.abs(ypred - y))
    #r2 = r2_score(y, ypred)
    #return (mae, r2)

# create function to put in loop
#def train_and_test(hidden_layer, activation_function, solver, alpha, max_iteration):
    #nnet = MLPRegressor(hidden_layer_sizes=hidden_layer, activation=activation_function, solver=solver,
                    #alpha=alpha, max_iter=max_iteration) # optimization
    #scores = cross_val_score(nnet, X, y, cv=5, scoring="r2") # cross-validation and calculate r^2 within
    #print(scores)
    #nnet.fit(Xtrain, ytrain)
    #print(nnet.predict(Xtest))


#for hidden_layer in hidden_layers:
    #for activation_function in activation_functions:
        #for solver in solvers:
            #for alpha in alphas:
                #for max_iteration in max_iterations:
                    #train_and_test(hidden_layer, activation_function, solver, alpha, max_iteration)

# train_and_test((32, 64, 32), "relu", "sgd", 0.001, 1000)

In [ ]:
scorers = ["r2", "neg_mean_absolute_error"]
parameters= {
    "hidden_layer_sizes": [(32, 64, 32), (64, 128, 64), (8, 16, 32, 16, 8), (32, 64, 128, 64, 32), (15, 27, 40, 31, 12)],
    "activation": ["relu", "logistic", "tanh", "identity"],
    "solver": ["sgd", "adam"],
    "alpha": [0.0001, 0.0005, 0.00005],
    "max_iter": [1000, 2000]
}

model = GridSearchCV(MLPRegressor(), parameters, cv=5, scoring=scorers, refit=False)
model.fit(X, y)

results = pd.DataFrame()
params = model.cv_results_["params"]

maes = model.cv_results_["mean_test_neg_mean_absolute_error"]
r2s = model.cv_results_["mean_test_r2"]

results["Params"] = pd.Series(params)
results["MeanMAE"] = -pd.Series(maes)
results["MeanR2"] = pd.Series(r2s)

results.sort_values("MeanR2", inplace=True)

results.to_csv("results_nnet.csv", sep=";", index=False)

results